In [25]:
from pathlib import Path
import sys
import pandas as pd
import numpy as np
from IPython.display import display




NA_VALUES = ["---", "--", "", " ", "NA", "N/A"]
# =========================
# FILE FINDER
# =========================
def find_file(name, start=Path.cwd()):
    for ancestor in [start] + list(start.parents):
        matches = list(ancestor.rglob(name))
        if matches:
            return matches[0]
    return None


def find_files(file_map):
    found = {}
    for key, filename in file_map.items():
        path = find_file(filename)
        if path:
            found[key] = path
        else:
            print(f"[WARNING] File not found: {filename}")
    return found

eda_script_path = find_file("script_eda.py")
if eda_script_path is None:
    raise FileNotFoundError("❌ script_eda.py tidak ditemukan di parent directory")

# tambahkan BASE PROJECT ke sys.path
sys.path.append(str(eda_script_path.parent))

# sekarang bisa import
from script_eda import evaluate_dataset, extract_column_schema,find_internal_duplicate_columns,extract_single_schema,cek_value_data_column


In [26]:
path_ispu = find_file("ispu_all_years.csv")

if path_ispu is None:
    raise FileNotFoundError("❌ File merged tidak ditemukan")

df_ispu = pd.read_csv(path_ispu, na_values=NA_VALUES)

df_copy = df_ispu.copy()

df_copy.head()



,periode_data,tanggal,stasiun,pm_sepuluh,pm_duakomalima,sulfur_dioksida,karbon_monoksida,ozon,nitrogen_dioksida,max,parameter_pencemar_kritis,kategori,id
0,201001,2010-01-19,DKI2 (Kelapa Gading),NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,TIDAK ADA DATA,2010-01-19_DKI2
1,201001,2010-01-20,DKI2 (Kelapa Gading),NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,TIDAK ADA DATA,2010-01-20_DKI2
2,201001,2010-01-21,DKI2 (Kelapa Gading),NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,TIDAK ADA DATA,2010-01-21_DKI2
3,201001,2010-01-22,DKI2 (Kelapa Gading),NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,TIDAK ADA DATA,2010-01-22_DKI2
4,201001,2010-01-23,DKI2 (Kelapa Gading),NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,TIDAK ADA DATA,2010-01-23_DKI2


In [27]:
count_tidak_ada_data = (df_copy["kategori"] == "TIDAK ADA DATA").sum()

print("Jumlah row 'TIDAK ADA DATA':", count_tidak_ada_data)

row_awal = df_copy.shape[0]
print("Row awal :", row_awal)

df_copy = df_copy[df_copy["kategori"] != "TIDAK ADA DATA"]

row_akhir = df_copy.shape[0]
print("Row akhir:", row_akhir)

if row_awal - row_akhir == count_tidak_ada_data:
    print("✅ Drop berhasil & konsisten")
else:
    print("❌ Ada ketidaksesuaian jumlah row")

assert not (df_copy["kategori"] == "TIDAK ADA DATA").any(), \
       "❌ Masih ada kategori 'TIDAK ADA DATA' tersisa"


Jumlah row 'TIDAK ADA DATA': 1458
Row awal : 16902
Row akhir: 15444
✅ Drop berhasil & konsisten


In [28]:
NUMERIC_COLS = [
    "pm_sepuluh",
    "pm_duakomalima",
    "sulfur_dioksida",
    "karbon_monoksida",
    "ozon",
    "nitrogen_dioksida"
]


df_work = df_copy.copy()

df_work["tanggal"] = pd.to_datetime(df_work["tanggal"], errors="coerce")

df_work = df_work.sort_values("tanggal").reset_index(drop=True)

df_work[NUMERIC_COLS] = df_work[NUMERIC_COLS].apply(
    pd.to_numeric,
    errors="coerce"
)

df_work = df_work.set_index("tanggal")

df_work[NUMERIC_COLS] = df_work[NUMERIC_COLS].interpolate(
    method="time"
)

df_work = df_work.reset_index()

print("Sisa NaN per kolom:")
print(df_work[NUMERIC_COLS].isna().sum())




Sisa NaN per kolom:
pm_sepuluh              0
pm_duakomalima       8439
sulfur_dioksida         0
karbon_monoksida        0
ozon                    0
nitrogen_dioksida       0
dtype: int64


In [29]:
# pastikan kolom ada
if "pm_duakomalima" in df_work.columns:
    df_work = df_work.drop(columns=["pm_duakomalima"])
    print("✅ Kolom 'pm_duakomalima' berhasil di-drop")
else:
    print("⚠️ Kolom 'pm_duakomalima' tidak ditemukan")

print("Kolom tersisa:")
print(df_work.columns.tolist())


✅ Kolom 'pm_duakomalima' berhasil di-drop
Kolom tersisa:
['tanggal', 'periode_data', 'stasiun', 'pm_sepuluh', 'sulfur_dioksida', 'karbon_monoksida', 'ozon', 'nitrogen_dioksida', 'max', 'parameter_pencemar_kritis', 'kategori', 'id']


In [30]:


OUTPUT_DIR = Path.cwd()
output_path = OUTPUT_DIR / "ispu_all_years_interpolated.csv"


df_work.to_csv(output_path, index=False)

print(f"✅ File berhasil disimpan di:\n{output_path}")

✅ File berhasil disimpan di:
c:\Users\USER\Desktop\DATAVIDIA\penyisihan-datavidia-10\ispu_named copy\ispu_all_years_interpolated.csv
